### Create ImplementationGuide Resource from an excel based configuration file
python 3.7+

- Load Excel file using Pandas and access as dict
- Assign appropriate fields to IG resource ( including defaults )
- Add resources and pages based on the files in the target ig folders
- Validate resource
- Write resource to target ig folder

### Imports and globals

In [3]:
import os #os module imported here
from json import load, dumps, loads
from requests import get, post, put
from IPython.display import display, Markdown, HTML
from pathlib import Path
from pandas import *
from datetime import datetime, date
from stringcase import snakecase, titlecase
from fhirclient.r4models import implementationguide as IG
import fhirclient.r4models.identifier as I
import fhirclient.r4models.coding as C
import fhirclient.r4models.codeableconcept as CC
import fhirclient.r4models.fhirdate as D
import fhirclient.r4models.extension as X
import fhirclient.r4models.contactdetail as CD
import fhirclient.r4models.fhirreference as FR
import ig_pages_template as p

#Globals

R4fhir_server = 'http://hapi.fhir.org/baseR4'
ig_source_path = '.'
headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }

f_now = D.FHIRDate(str(date.today()))

p.pages

{'grouping': [{'name': 'base'}],
 'page': {'nameUrl': 'index.html',
  'title': 'Home',
  'generation': 'markdown',
  'page': [{'nameUrl': 'guidance.html',
    'title': 'General Guidance',
    'generation': 'markdown'},
   {'nameUrl': 'profiles.html',
    'title': 'Profiles and Extensions',
    'generation': 'markdown'},
   {'nameUrl': 'operations.html',
    'title': 'Operations',
    'generation': 'markdown'},
   {'nameUrl': 'terminology.html',
    'title': 'Terminology',
    'generation': 'markdown'},
   {'nameUrl': 'searchparameters.html',
    'title': 'Search Parameters',
    'generation': 'markdown'},
   {'nameUrl': 'capstatements.html',
    'title': 'Capability Statements',
    'generation': 'markdown'},
   {'nameUrl': 'security.html', 'title': 'Security', 'generation': 'markdown'},
   {'nameUrl': 'downloads.html',
    'title': 'Downloads',
    'generation': 'markdown'},
   {'nameUrl': 'all-examples.html',
    'title': 'All Examples',
    'generation': 'markdown'},
   {'nameUrl': 

### Get definition spreadsheet from folder

- Use the generated spreadsheet from the FHIR Specification
- right now this is hard coded location - make a user input
- import data from spreadsheet
- convert to pandas df and then dict

In [4]:
def_path='.'

xls = ExcelFile(f'{def_path}/implementationguide.xlsx')
df = read_excel(xls,'Elements', na_filter=False)

df

,Path,Element,Example,Slice Name,Alias(s),Label,Min,Max,Must Support?,Is Modifier?,...,Slicing Rules,Base Path,Base Min,Base Max,Condition(s),Constraint(s),Mapping: RIM Mapping,Mapping: Workflow Pattern,Mapping: FiveWs Pattern Mapping,Mapping: Object Implementation Information
0,ImplementationGuide,NaN,,,,,0,*,,,...,,ImplementationGuide,0,*,,dom-2:If the resource is contained in another ...,n/a,,,
1,ImplementationGuide.id,id,,,,,0,1,,,...,,Resource.id,0,1,,,,,,
2,ImplementationGuide.meta,meta,,,,,0,1,,,...,,Resource.meta,0,1,,,,,,
3,ImplementationGuide.url,url,,,url\nauthoritative-urldestinationidentity,,1,1,,,...,,ImplementationGuide.url,1,1,,,,Definition.url,FiveWs.identifier,
4,ImplementationGuide.version,version,0.0.0,,,,0,1,,,...,,ImplementationGuide.version,0,1,,,,Definition.version,FiveWs.version,
5,ImplementationGuide.name,name,FooBar,,,,1,1,,,...,,ImplementationGuide.name,1,1,inv-0\n,,,,,
6,ImplementationGuide.title,title,,,,,0,1,,,...,,ImplementationGuide.title,0,1,,,,Definition.title,,
7,ImplementationGuide.status,status,,,,,1,1,,Y,...,,ImplementationGuide.status,1,1,,,,Definition.status,FiveWs.status,
8,ImplementationGuide.experimental,experimental,,,,,0,1,,,...,,ImplementationGuide.experimental,0,1,,,,Definition.experimental,FiveWs.class,
9,ImplementationGuide.date,date,,,Revision Date\n,,0,1,,,...,,ImplementationGuide.date,0,1,,,,Definition.date,FiveWs.recorded,


In [5]:
d = dict(zip(df.Element, df.Example))
d

{nan: '',
 'id': '',
 'meta': '',
 'url': '',
 'version': '0.0.0',
 'name': 'FooBar',
 'title': '',
 'status': '',
 'experimental': '',
 'date': '',
 'publisher': 'Health eData Inc',
 'contact': '',
 'contact.system': 'email',
 'contact.value': 'mailto: ehaas@healthedatainc.com',
 'description': '',
 'useContext': '',
 'jurisdiction': '',
 'copyright': '',
 'packageId': 'foo.bar',
 'license': 'yes',
 'fhirVersion': '1.0.0',
 'definition': '',
 'dependsOn': 'see tab',
 'global': 'see tab',
 'definition.grouping': '',
 'definition.grouping.name': '',
 'definition.grouping.description': '',
 'definition.resource': 'see tab',
 'definition.page': '',
 'definition.page.name[x]': '',
 'definition.page.title': '',
 'definition.page.generation': '',
 'definition.page.page': '',
 'definition.parameter': 'see tab',
 'definition.template': 'see tab',
 'manifest': '',
 'manifest.rendering': '',
 'manifest.resource': '',
 'manifest.resource.reference': '',
 'manifest.resource.example[x]': '',
 'mani

### Assign appropriate fields to IG resource ( including defaults )
- Initiate ig resource instance
- assign id and defaults

In [6]:
ig = IG.ImplementationGuide()
ig.id = f'{d["packageId"]}-{d["version"]}'
ig.url = f'{d["packageId"].replace(".","/")}-{d["version"]}'
ig.name = d["name"]
ig.title = f'{titlecase(d["name"])} Implementation Guide'
ig.status = d['status'] if d['status'] else 'active'
ig.date = f_now
ig.publisher = d['publisher'] if d['publisher'] else None

publisher_endpoint = dict(
                         system= d['contact.system'] if d['contact.system'] else None,
                         value = d['contact.value'] if d['contact.value'] else None,
                        )
ig.contact =  [CD.ContactDetail( {"telecom" : [ publisher_endpoint ] })]
              
ig.copyright = d['copyright'] if d['copyright'] else None
ig.packageId = d['packageId'] if d['packageId'] else None
ig.license = d['license'] if d['license'] else None

ig.fhirVersion = [d['fhirVersion']]  # only one for now

ig.as_json()

{'id': 'foo.bar-0.0.0',
 'contact': [{'telecom': [{'system': 'email',
     'value': 'mailto: ehaas@healthedatainc.com'}]}],
 'date': '2019-06-19',
 'fhirVersion': ['1.0.0'],
 'license': 'yes',
 'name': 'FooBar',
 'packageId': 'foo.bar',
 'publisher': 'Health eData Inc',
 'status': 'active',
 'title': 'Foo Bar Implementation Guide',
 'url': 'foo/bar-0.0.0',
 'resourceType': 'ImplementationGuide'}

### Add resources and pages based on the files in the target ig folders

- start with fixed group and pages as string template
- for all files in folders inspect and add
- using lmxml to get data for spreadsheets and xml files

In [33]:
ig.definition = IG.ImplementationGuideDefinition(p.pages, strict=False)

rr = FR.FHIRReference(dict(
          reference = "foo/bar.json",
          display  = "Foo Bar"
            )
           )

#print(rr.as_json())

ig.definition.resource = []
ig_resource = IG.ImplementationGuideDefinitionResource()
ig_resource.reference = rr
ig.definition.resource.append(ig_resource)

print(dumps(ig.as_json(), indent=4))

{
    "id": "foo.bar-0.0.0",
    "contact": [
        {
            "telecom": [
                {
                    "system": "email",
                    "value": "mailto: ehaas@healthedatainc.com"
                }
            ]
        }
    ],
    "date": "2019-06-19",
    "definition": {
        "grouping": [
            {
                "name": "base"
            }
        ],
        "page": {
            "generation": "markdown",
            "nameUrl": "index.html",
            "page": [
                {
                    "generation": "markdown",
                    "nameUrl": "guidance.html",
                    "title": "General Guidance"
                },
                {
                    "generation": "markdown",
                    "nameUrl": "profiles.html",
                    "title": "Profiles and Extensions"
                },
                {
                    "generation": "markdown",
                    "nameUrl": "operations.html",
                 

### Validate resource using the $validate operation

- using UHN R4 reference server

In [28]:
r =  post(f'{R4fhir_server}/ImplementationGuide/$validate', headers = headers, data = dumps(ig.as_json()).encode('utf-8'))   # return r.status_code
display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))

ERROR,[],"Failed to parse request body as JSON resource. Error was: Invalid attribute value ""yes"": Unknown SPDXLicense code 'yes'"


### Write resource to target ig folder

- use Pathlib here

In [30]:
print('...........saving to file............')
path = Path.cwd() / ig_source_path / 'resources' / f'implementationguide-{ig.id.lower()}.json' #save in ig_source folder
path.write_text(dumps(ig.as_json(), indent=4))

...........saving to file............


2844